In [1]:
import torch
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data_utils

In [2]:
conv1 = nn.Conv2d(1,  32, 3, padding = 1) #input ch, output ch, kernel size, stride, padding
conv2 = nn.Conv2d(32, 64, 3, 1, padding = 1)

In [3]:
print(conv1)
print(conv2)

# stride 에 아무런 값을 안주면 디폴트로 1이 들어감을 알 수 있다

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


## ☆실제로 CNN 의 과정을 살펴보자☆

In [4]:
input_data = torch.Tensor(1,1,28,28) # MNIST 의 실제 shape
print(input_data.shape)

torch.Size([1, 1, 28, 28])


$$shape = \displaystyle \frac{(Input shape) - (kernel size)+ 2*(padding)}{stride} + 1$$

input shape = (28,28) 이고 <br>
첫번째 conv layer 를 통과하면 (28,28) - (3,3) + (2,2) + (1,1) = (28,28)<br>
즉, channel 갯수만 늘고, shape 는 변하지 않음을 알 수 있다

In [5]:
print(conv1(input_data).shape)

torch.Size([1, 32, 28, 28])


nn.MaxPool2d(2) 의 경우 kernel size 가 2 라면 stride 도 2로 고정되어있다.<br>
padding 은 물론 0<br><br>
즉<br>
((28,28) - (2,2)) / 2 후에 + (1,1) 은 (14,14)

In [6]:
pool = nn.MaxPool2d(2)
print(pool(conv1(input_data)).shape)

torch.Size([1, 32, 14, 14])


같은 방식으로 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) 는 어떨까?

- (14,14) 에서 (3,3) 을 빼고 (2,2) 를 더한 후 다시 (1,1) 을 더하면 그대로 (14,14) 이다.

In [7]:
final_output = conv2(pool(conv1(input_data)))
print(final_output.shape)

torch.Size([1, 64, 14, 14])


In [8]:
## CNN 의 output 을 linear layer 에 통과시켜줄 차례
## 배치 사이즈는 그대로 두고, 나머지는 한줄로 펼치자
out = final_output.view(final_output.shape[0], -1)
print(out.shape)

torch.Size([1, 12544])


In [9]:
fc = nn.Linear(12544,10)

In [10]:
fc(out).shape

torch.Size([1, 10])

### 본격적인 CNN 시작

In [11]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)
    
print(device)

cuda


In [12]:
from mnist import MNIST
import numpy as np

# MNIST library 를 MNIST image 파일이 있는 path 를 통해서 불러온 후 mnist 변수에 담기
mnist = MNIST('../image_data/MNIST/MNIST/raw')

# x_train, y_train, x_test, y_test 로 나누어준다
x_train, y_train = mnist.load_training()
x_test, y_test = mnist.load_testing()

# data 는 list 형식을 되어 있으므로 shape 을 보고, 이미지 visualization 을 하기 편한 array 형태로 바꾸어준다.
x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
x_test=np.asarray(x_test)
y_test=np.asarray(y_test)

print("x_train 의 shape={}, y_train 의 shape={}".format(x_train.shape,y_train.shape))
print("x_test 의 shape={}, y_test 의 shape={}".format(x_test.shape,y_test.shape))

x_train 의 shape=(60000, 784), y_train 의 shape=(60000,)
x_test 의 shape=(10000, 784), y_test 의 shape=(10000,)


### Convert array data into Tensor Dataset

1. array 형태의 x와 y data 를 TensorDataset 형태로 train_data 에 담고
2. 정해진 Batch size 를 이용해서 data 를 load 한 후
3. 모델을 짜준 후에 training

In [13]:
# x data 와 y data 를 하나로 합침
train_data = data_utils.TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
batch_size = 1000

# batch size 별로 가져올 수 있게 data load
trainloader = data_utils.DataLoader(train_data, batch_size = batch_size, shuffle = True)

#### CNN 모델을 스스로 구현해보자

  (layer1): Sequential( <br>
    (0): Conv Layer (kernel size = 1, stride = 1, padding = 1) <br>
    (1): Batch Normalization<br>
    (2): ReLU <br>
    (3): MaxPooling (kernel size = 2) <br>
<br>
  (layer2): Sequential( <br>
    (0): Conv Layer (kernel size = 5, stride = 2, padding = 0) <br>
    (1): Batch Normalization<br>
    (2): ReLU <br>
    (3): MaxPooling (kernel size = 2) <br>
<br>
  (fc): Linear(in_features=???, out_features=10) <br>

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN ,self).__init__()
        
        # -> CONV/FC -> BatchNorm -> ReLu(or other activation) -> Dropout -> CONV/FC ->
        # https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout
    
        self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 1, stride = 1, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 5, stride = 2, padding = 0),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        
        self.fc = nn.Linear(3*3*128,10)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        
        return out

### cf) Convolutional Layer 의 output 을 쉽게 구할 수 있는 방법

Conv layer 를 지나고 나서 과연 어떤 size 의 tensor data 가 나올지 손쉽게 계산해 볼 수 있는 방법은 없을까? <br>
한가지 팁으로 dummy data를 넣어보는 방법을 소개한다.

방법은 간단하다.  
dummy data를 input image 와 같은 size 로 만들고 output size 를 알아보면 된다.<br>
물론 이때 사이즈는 28 \* 28 이어야한다

In [15]:
dummy_data = torch.Tensor(1000,1,28,28).to(device) # 1000 은 bath size 를 나타냄
print(dummy_data.shape)

torch.Size([1000, 1, 28, 28])


중간에 conv layer 를 지나고 난 후 shape 를 확인 할 수 있도록 Conv Layer 까지만 model 을 짜본다

In [16]:
class dummy_CNN(nn.Module):
    def __init__(self):
        super(dummy_CNN ,self).__init__()
        
        # -> CONV/FC -> BatchNorm -> ReLu(or other activation) -> Dropout -> CONV/FC ->
        # https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout
    
        self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 1, stride = 1, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 5, stride = 2, padding = 0),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        return out
    
dummy_model = dummy_CNN().to(device)

아래를 통해 Conv Layer의 output 은 1000(batch size) * 128(output chennel) * 3 * 3 을 나타냄을 알 수 있다

In [17]:
dummy_model(dummy_data).shape

torch.Size([1000, 128, 3, 3])

즉, Linear layer 의 input 은 batch size 를 제외한 128 * 3 * 3 이 되어야한다!

다시 CNN network 를 만들어보자

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN ,self).__init__()        
        self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 1, stride = 1, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 5, stride = 2, padding = 0),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2))
        
        self.fc = nn.Linear(3*3*128,10)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        
        return out

In [19]:
model = CNN().to(device)

##### Parameters

In [20]:
epoch = 15
learning_rate = 0.001
weight_decay = 1e-5

In [21]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

##### Model training 의 순서를 기억하시나요?
1. x_data (image), y_data (label) 을 나누는 것
2. gpu 메모리 위에 올려놓기
3. gradient 0 으로 초기화
4. model 에 data 를 넣어서 prediction 값 도출
5. loss function 을 이용해 loss 값 구하기
6. backpropagation
7. weight update

In [22]:
# training

total_batch = len(trainloader) # 전체 mini batch 의 갯수

for num_epoch in range(epoch):
    avg_loss = 0
    
    for batch_num, (images, labels) in enumerate(trainloader):
        # 1. x_data (image), y_data (label) 을 나누는 것
        # 2. gpu 메모리 위에 올려놓기
        X = images.to(device)
        X = X.reshape(1000,1,28,28) # 1000 = batch size

        #Y = torch.tensor(labels, dtype = torch.long)
        Y = labels.to(device)
        
        # 3. gradient 0 으로 초기화
        optimizer.zero_grad()
        
        # 4. model 에 data 를 넣어서 prediction 값 도출
        predict = model(X)
        
        # 5. loss function 을 이용해 loss 값 구하기
        loss = loss_function(predict, Y.long())
        
        # 6. backpropagation
        loss.backward()
        
        # 7. weight update
        optimizer.step()
        
        avg_loss = avg_loss + (loss/total_batch)
        
    print("Epoch = {} loss = {:.4f}".format(num_epoch+1, avg_loss.item()))

Epoch = 1 loss = 0.6361
Epoch = 2 loss = 0.1987
Epoch = 3 loss = 0.1403
Epoch = 4 loss = 0.1108
Epoch = 5 loss = 0.0937
Epoch = 6 loss = 0.0821
Epoch = 7 loss = 0.0724
Epoch = 8 loss = 0.0664
Epoch = 9 loss = 0.0627
Epoch = 10 loss = 0.0565
Epoch = 11 loss = 0.0524
Epoch = 12 loss = 0.0490
Epoch = 13 loss = 0.0479
Epoch = 14 loss = 0.0445
Epoch = 15 loss = 0.0424


In [23]:
torch.save(model.state_dict(), './pre_trained/CNN_MNIST.pth')

In [24]:
model = CNN().to(device)

In [25]:
model.load_state_dict(torch.load('./pre_trained/CNN_MNIST.pth', map_location=device))

<All keys matched successfully>

### Model Test

In [26]:
# x data 와 y data 를 하나로 합침
test_data = data_utils.TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(y_test))
batch_size = 10000

# batch size 별로 가져올 수 있게 data load
testloader = data_utils.DataLoader(train_data, batch_size = batch_size, shuffle = False)

In [28]:
# test
with torch.no_grad():
    num_total_data = 0
    correct = 0
    
    for batch_idx, (images, labels) in enumerate(testloader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        images = images.reshape(batch_size,1,28,28) # lbatch_size = 10000
        
        outputs = model(images).to(device)
        outputs_softmax= torch.nn.functional.softmax(outputs, dim=1)
        
        # softmax 를 이용해 probability 가 가장 큰 index 를 가져옴
        predicted = torch.argmax(outputs_softmax, dim=1)
        
        # len(images) 는 결국 배치 size 를 나타내는 것이므로 batch size 를 계속 더해주면 data 의 총 길이가 된다.
        num_total_data = num_total_data + len(images)
        
        # 맞게 예측한 것만 세어야하는데 이때 쓸 수 있는 좋은 방법은 (True is equal to 1) 를 이용하는 것이다.
        # itme() 을 해주면 tensor type 을 벗어던지고 단순한 float 형으로 다시 태어날 수 있다.
        
        answer = sum(labels==predicted).item()       
        correct = correct + answer
        
print("CNN 을 이용한 모델의 정확도는 {:.5}%".format((correct/num_total_data)*100))

CNN 을 이용한 모델의 정확도는 99.082%
